# Third Point

Use transfer learning based on a pre-trained network in two manners.

In [1]:
import cv2 as cv2
import glob
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D,Flatten,Dropout, MaxPooling2D,AveragePooling2D
from keras import optimizers, initializers
from keras.callbacks import EarlyStopping, History
from keras.applications.vgg16 import VGG16
from sklearn.metrics import confusion_matrix
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

import import_ipynb
import dataSetUtility as dsu
import plottingUtility as pltu

importing Jupyter notebook from dataSetUtility.ipynb
importing Jupyter notebook from plottingUtility.ipynb


Read all images and create a Training Data

In [2]:
#path to the directories
pathTrain=r'C:\Users\adria\Desktop\CVPR\ImageSet\train'
pathTest=r'C:\Users\adria\Desktop\CVPR\ImageSet\test'

labels = [os.path.basename(i) for i in glob.glob(pathTrain + '/*', recursive=True)]
numberOfClasses = len(labels)
print("Class number: ", numberOfClasses)
print("Class names: ", labels)

Class number:  15
Class names:  ['Bedroom', 'Coast', 'Forest', 'Highway', 'Industrial', 'InsideCity', 'Kitchen', 'LivingRoom', 'Mountain', 'Office', 'OpenCountry', 'Store', 'Street', 'Suburb', 'TallBuilding']


To use images on VGG they need to have size (224, 224)

In [3]:
xTrain, yTrain = dsu.loadImages(pathTrain, labels, dimension=224)

create a list of images for train and test set

In [6]:
trainImagesList = []
testImagesList = []
for label in labels:
    sTrain = pathTrain+'/'+label+'/{}'
    sTest = pathTest+'/'+label+'/{}'
    tempTrain = [sTrain.format(i) for i in os.listdir(pathTrain+'/'+label+'/')]
    tempTest = [sTest.format(i) for i in os.listdir(pathTest+'/'+label+'/')]
    trainImagesList.append(tempTrain)
    testImagesList.append(tempTest)